# Edge Map Generation (Line Art)

This notebook generates edge (line art) maps for all images in the `balanced_sample_2k_512x512_sketches` folder and saves them into `balanced_sample_2k_512x512_maps`.

Detector: `LineartDetector.from_pretrained("lllyasviel/Annotators")`

Features:
- Skips files already processed
- Progress bar with ETA
- Batch-friendly (adjustable)
- Safety checks (corrupt image handling)
- Optional preview sampling

> All outputs are 512x512 PNG files (single-channel or 3-channel depending on detector output).

In [1]:
# Imports & Environment Setup
import os
import sys
from pathlib import Path
from typing import List
import math
import traceback
from datetime import datetime

import PIL
from PIL import Image
from tqdm import tqdm

try:
    from controlnet_aux import LineartDetector
except ImportError:
    raise ImportError("controlnet-aux not installed. Install via: pip install controlnet-aux --upgrade")

# 2. Configuration for batch sketch generation
# Root (adjust if running from another location)
PROJECT_ROOT = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
INPUT_DIR = PROJECT_ROOT / '../../output/amazing_logos_v4/images/balanced_sample_2k_512x512_sketches_postproc'
OUTPUT_DIR = PROJECT_ROOT / '../../output/amazing_logos_v4/images/balanced_sample_2k_512x512_maps2'

INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Input dir: {INPUT_DIR}")
print(f"Output dir: {OUTPUT_DIR}")

c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\controlnet_aux\mediapipe_face\mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


Project root: c:\studium\master_thesis\data_prep\notebooks\sktech_creation
Input dir: c:\studium\master_thesis\data_prep\notebooks\sktech_creation\..\..\output\amazing_logos_v4\images\balanced_sample_2k_512x512_sketches_postproc
Output dir: c:\studium\master_thesis\data_prep\notebooks\sktech_creation\..\..\output\amazing_logos_v4\images\balanced_sample_2k_512x512_maps2


c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\paul-\miniconda3\envs\py311-cuda\Lib\site-packages\controlnet_aux\segment_anything\modeling\tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\Users\paul-\m

In [2]:
# Initialize Lineart Detector
# Adjust model variant if needed. The repository 'lllyasviel/Annotators' bundles multiple annotators.
# LineartDetector supports optional parameters like 'coarse' or 'resolution' depending on version.

detector = LineartDetector.from_pretrained("lllyasviel/Annotators")
print("Detector loaded:", type(detector))

Detector loaded: <class 'controlnet_aux.lineart.LineartDetector'>


In [3]:
# Utilities
SUPPORTED_EXTS = {'.png', '.jpg', '.jpeg', '.webp'}

def list_images(folder: Path) -> List[Path]:
    return sorted([p for p in folder.iterdir() if p.suffix.lower() in SUPPORTED_EXTS and p.is_file()])

def load_image(path: Path) -> Image.Image:
    with Image.open(path) as im:
        im = im.convert('RGB')
        return im.copy()

def save_image(img: Image.Image, path: Path):
    img.save(path, format='PNG')

def compute_lineart(img: Image.Image):
    # Detector may accept PIL Image or numpy array depending on version
    result = detector(img)
    if isinstance(result, Image.Image):
        return result
    # Try to coerce numpy array to PIL
    try:
        import numpy as np
        if isinstance(result, (list, tuple)):
            result = result[0]
        if isinstance(result, np.ndarray):
            if result.ndim == 2:
                return Image.fromarray(result)
            if result.ndim == 3:
                return Image.fromarray(result[:, :, :3])
    except Exception:
        pass
    raise TypeError(f"Unexpected detector output type: {type(result)}")

In [4]:
# Main processing loop
from collections import Counter

images = list_images(INPUT_DIR)
print(f"Found {len(images)} input images.")

processed = 0
skipped = 0
errors: List[str] = []

for img_path in tqdm(images, desc='Generating lineart maps'):
    out_path = OUTPUT_DIR / (img_path.stem + '.png')
    if out_path.exists():
        skipped += 1
        continue
    try:
        img = load_image(img_path)
        edge = compute_lineart(img)
        # Ensure consistent mode (convert to L or keep as is?)
        if edge.mode not in ('L', 'RGB'):
            edge = edge.convert('L')
        save_image(edge, out_path)
        processed += 1
    except Exception as e:
        errors.append(f"{img_path.name}: {e}")
        traceback.print_exc()

print(f"Done. New: {processed}, Skipped(existing): {skipped}, Errors: {len(errors)}")
if errors:
    print('\nSample errors (first 5):')
    for line in errors[:5]:
        print(' -', line)

Found 1810 input images.


Generating lineart maps: 100%|██████████| 1810/1810 [07:00<00:00,  4.30it/s]

Done. New: 1810, Skipped(existing): 0, Errors: 0


In [5]:
# Optional: Preview some results
import random
from IPython.display import display

sample = random.sample(images, k=min(5, len(images)))
for p in sample:
    out_p = OUTPUT_DIR / (p.stem + '.png')
    if not out_p.exists():
        continue
    orig = load_image(p)
    edge = load_image(out_p)
    display({'Original': orig, 'EdgeMap': edge})

{'Original': <PIL.Image.Image image mode=RGB size=512x512>,
 'EdgeMap': <PIL.Image.Image image mode=RGB size=512x512>}

{'Original': <PIL.Image.Image image mode=RGB size=512x512>,
 'EdgeMap': <PIL.Image.Image image mode=RGB size=512x512>}

{'Original': <PIL.Image.Image image mode=RGB size=512x512>,
 'EdgeMap': <PIL.Image.Image image mode=RGB size=512x512>}

{'Original': <PIL.Image.Image image mode=RGB size=512x512>,
 'EdgeMap': <PIL.Image.Image image mode=RGB size=512x512>}

{'Original': <PIL.Image.Image image mode=RGB size=512x512>,
 'EdgeMap': <PIL.Image.Image image mode=RGB size=512x512>}